In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.5 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 16.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 11.4 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 63.0 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 32.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 9.8 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-cbpl87ys/overlay/local/bin
    changing mode of /tmp/pip-build-env-cbpl87ys/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-cbpl87ys/overlay/

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [3]:
from huggingface_hub import hf_hub_download

In [4]:
from llama_cpp import Llama

In [5]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [6]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [7]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

In [29]:
prompt = "This statement shows your account activity for the month of January. Your balance as of January 31st is $1,234.56."

prompt_template= f'''SYSTEM:** I am a text classification assistant. I analyzed the statement and predict the most likely class of that statement:

*Predicted Class (One Word):*

USER: {prompt}
'''

In [30]:
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

Llama.generate: prefix-match hit


In [31]:
print(response["choices"][0]["text"])

SYSTEM:** I am a text classification assistant. I analyzed the statement and predict the most likely class of that statement:

*Predicted Class (One Word):*

USER: This statement shows your account activity for the month of January. Your balance as of January 31st is $1,234.56.

Please let me know if you have any questions or concerns about this information.

SYSTEM: Based on my analysis, I predict that the most likely class of this statement is "Invoice". Is there anything else I can assist with?


In [11]:
print(response)

{'id': 'cmpl-643c2dfb-cb6e-4768-bd0b-68bf8c614238', 'object': 'text_completion', 'created': 1708865639, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:** I am a document classification assistant. I analyzed the document and predict the most likely class:\n\nMedical Document:** Contains information about health, diagnoses, or treatments.\nFinancial/Legal Document:** Deals with money, contracts, or legal matters.\nCertificate:** Official document verifying something (e.g., birth, completion of a course).\n\nUSER: This statement shows your account activity for the month of January. Your balance as of January 31st is $1,234.56.\n\nPlease select one of the following options to classify this document:\n\nA) Medical Document\nB) Financial/Legal Document\nC) Certificate\n\nWhat would you like to do?', 'index': 0, 'logprobs': None, 'finish_reason': '

In [ ]:
# import re  # For regular expression extraction

# prompt = "This statement shows your account activity for the month of January. Your balance as of January 31st is $1,234.56."

# # Modify the prompt template with explicit instructions and simplified options:
# prompt_template = f'''SYSTEM: You are a document classifier that predicts the class of the input document.:

# Financial/Legal Document (Deals with money, contracts, or legal matters)
# Medical Document (Contains information about health, diagnoses, or treatments)
# Certificate (Official document verifying something)

# USER: {prompt}

# ASSISATANT:
# '''

# # Set higher temperature and adjust top_k for more creative output:
# response = lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_k=50, top_p=0.95,
#                   repeat_penalty=1.2, echo=True)

# print(f'Predicted Class (One word): {response["choices"][0]["text"]}')
